In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 33 (delta 15), reused 26 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 8.20 KiB | 2.73 MiB/s, done.
Resolving deltas: 100% (15/15), done.
/kaggle/working/agnews-classifier


In [11]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=3,
    lora_alpha=6,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'key'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 704,260 || all params: 125,352,968 || trainable%: 0.5618


In [12]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [13]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.370400,1.344790,0.698438
200,1.307900,1.235629,0.845313
300,1.062200,0.767385,0.876563
400,0.558000,0.382581,0.887500
500,0.376100,0.336637,0.889062
600,0.353900,0.329367,0.898438
700,0.328800,0.330740,0.898438
800,0.358600,0.319129,0.900000
900,0.347300,0.321294,0.901563
1000,0.333100,0.318958,0.898438


In [16]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-1600'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:58<00:00,  4.26it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-1600.csv


In [17]:
import os
import pandas as pd

# Define paths
csv_path = "/kaggle/working/saved_models/checkpoint-2000/log_history.csv"
output_dir = "/kaggle/working/processed_data"
output_csv = os.path.join(output_dir, "processed_log_history.csv")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the csv file
df = pd.read_csv(csv_path)

# Select desired columns and reorder the dataframe
desired_order = [
    "step",         
    "loss",         
    "eval_loss",   
    "accuracy",     
    "eval_accuracy",
    "learning_rate",
    "epoch"         
]
df = df[desired_order]

# Rename columns for uniformity
df.rename(columns={
    "step": "Step",
    "loss": "Train Loss",
    "eval_loss": "Test Loss",
    "accuracy": "Train Acc",
    "eval_accuracy": "Test Acc",
    "learning_rate": "Learning Rate",
    "epoch": "Epochs"
}, inplace=True)

# Adding loss spread and loss ratio columns
df["loss spread"] = df["Train Loss"] - df["Test Loss"]
df["loss ratio"] = df["Train Loss"] / df["Test Loss"]

# Adding acc spread and acc ratio columns
df["Acc spread"] = df["Test Acc"] - df["Train Acc"]
df["Acc ratio"] = df["Test Acc"] / df["Train Acc"]

# Export the DataFrame as a csv file
df.to_csv(output_csv, index=False)

print(f"Data exported to {output_csv}")

df

Data exported to /kaggle/working/processed_data/processed_log_history.csv


,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,loss spread,loss ratio,Acc spread,Acc ratio
0,100,1.3704,1.344790,0.360000,0.698438,0.000048,0.013405,0.025610,1.019044,0.338438,1.940104
1,200,1.3079,1.235629,0.624107,0.845313,0.000045,0.026810,0.072271,1.058490,0.221205,1.354435
2,300,1.0622,0.767385,0.817857,0.876563,0.000043,0.040214,0.294815,1.384182,0.058705,1.071779
3,400,0.5580,0.382581,0.883036,0.887500,0.000040,0.053619,0.175419,1.458514,0.004464,1.005056
4,500,0.3761,0.336637,0.888393,0.889062,0.000037,0.067024,0.039463,1.117228,0.000670,1.000754
5,600,0.3539,0.329367,0.889286,0.898438,0.000035,0.080429,0.024533,1.074484,0.009152,1.010291
6,700,0.3288,0.330740,0.890625,0.898438,0.000033,0.093834,-0.001940,0.994133,0.007812,1.008772
7,800,0.3586,0.319129,0.883929,0.900000,0.000030,0.107239,0.039471,1.123682,0.016071,1.018182
8,900,0.3473,0.321294,0.890179,0.901563,0.000028,0.120643,0.026006,1.080942,0.011384,1.012788
9,1000,0.3331,0.318958,0.895982,0.898438,0.000025,0.134048,0.014142,1.044339,0.002455,1.002740
